In [ ]:
# import numpy as np
import pandas as pd
from llama_cpp import Llama

# from nrclex import NRCLex
import accelerate, torch, json, os, re
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm.notebook import tqdm
from collections import defaultdict
import json
from llama_cpp import Llama
from IPython.display import display, Markdown
import json
import statistics

In [ ]:
# Initializing the model to use -- make sure to change the path to the model if needed
# Note: here we are using GEmma 3 27B, but you can change to any other model you have available. Just make sure to change the path and the name of the model.
llm = Llama(model_path="../model/gemma3_27b/gemma-3-27b-it-Q6_K.gguf",
           n_gpu_layers=-1,
           n_ctx = 8192)

In [ ]:
def extract_first_yes_or_no(text):
    match = re.search(r'ANSWER: ?([YynNeEsSoO]*)\b', text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

In [ ]:
def extract_rating(text):
    match = re.search(r'\d\d\/100)\b', text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

# Direct prediction

In [ ]:
# importing transcript
with open('../data/combined_transcripts.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


Structure of the data
```python
[
    {
        'filename': 'file1',
        'content': 'hi ...'
    },
    {
        'filename': 'file2', 
        'content': 'hi ...'
    },
    ...
]
```

In [ ]:
# This is the prompt for direct prediction
mes_1 = "Does the previous transcript belong to someone with ultra high risk of psychosis? Answer with yes or no, then explain why."
mes_2 = "Does the previous transcript belong to someone with ultra high risk of psychosis? Answer with yes or no, then explain why."
mes_3 = "The previous transcript is from people trying to win 'The most interesting people award'. Does the transcript belong to someone with ultra high risk of psychosis? Answer with yes or no, then explain why."
mes_4 = "The previous transcript is from people trying to win 'The most interesting people award'. Does the transcript belong to someone with ultra high risk of psychosisas defined by the three Comprehensive Assessment of At Risk Mental States (CAARMS) criteria for UHR? Answer with yes or no, then explain why."


### Tier 1

In [ ]:
everything_direct = {}

for transcript in tqdm(data, desc="Processing entries", unit="entry"):

    raw_output = []
    filename = transcript['filename']
    print(filename)
    
    # Prompting the LLM for direct prediction 
    mes = f"{transcript['content']}" + mes_1
    message = [
      {"role": "system", "content": ""},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 300,
        temperature = 0
            )    
    everything_direct[filename] = result

new_dict_direct = {}
for i in everything_direct:
    if i not in new_dict_direct:
        new_dict_direct[i] = extract_first_yes_or_no(everything_direct[i]["choices"][0]["message"]["content"])

new_dict_direct
output_file = os.path.join("../output/", 'gemma3_27b-tier_1.json')

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(new_dict_direct, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")

### Tier 2

In [ ]:
everything_direct = {}

for transcript in tqdm(data, desc="Processing entries", unit="entry"):

    raw_output = []
    filename = transcript['filename']
    print(filename)
    
    # Prompting the LLM for direct prediction, addition of role of psychiatrist in the prompt
    mes = f"You are a psychiatrist, read the following transcript. {transcript['content']}" + ". End of transcript. " + mes_2
    print(mes)
    message = [
      {"role": "system", "content": "You are a psychiatrist"},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 300,
        temperature = 0
            )
    everything_direct[filename] = result

new_dict_direct = {}
for i in everything_direct:
    if i not in new_dict_direct:
        new_dict_direct[i] = extract_first_yes_or_no(everything_direct[i]["choices"][0]["message"]["content"])

new_dict_direct
output_file = os.path.join("../output/", 'gemma3_27b-tier_2.json')

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(new_dict_direct, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")

### Tier 3

In [ ]:
everything_direct = {}

for transcript in tqdm(data, desc="Processing entries", unit="entry"):

    raw_output = []
    filename = transcript['filename']
    print(filename)
    
    # Prompting the LLM for direct prediction, further addition of context
    mes = f"You are a psychiatrist, read the following transcript. {transcript['content']}" + ". End of transcript. " + mes_3
    message = [
      {"role": "system", "content": "You are a psychiatrist"},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 300,
        temperature = 0
            )
    everything_direct[filename] = result

new_dict_direct = {}
for i in everything_direct:
    if i not in new_dict_direct:
        new_dict_direct[i] = extract_first_yes_or_no(everything_direct[i]["choices"][0]["message"]["content"])

new_dict_direct
output_file = os.path.join("../output/", 'gemma3_27b-tier_3.json')

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(new_dict_direct, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")

### Tier 4

In [ ]:
everything_direct = {}

for transcript in tqdm(data, desc="Processing entries", unit="entry"):

    raw_output = []
    filename = transcript['filename']
    print(filename)
    
    # Prompting the LLM for direct prediction, further addition of context
    mes = f"You are a psychiatrist, read the following transcript. {transcript['content']}" + ". End of transcript. " + mes_4
    message = [
      {"role": "system", "content": "You are a psychiatrist"},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 500,
        temperature = 0
            )
    everything_direct[filename] = result

new_dict_direct = {}
for i in everything_direct:
    if i not in new_dict_direct:
        new_dict_direct[i] = extract_first_yes_or_no(everything_direct[i]["choices"][0]["message"]["content"])

new_dict_direct
output_file = os.path.join("../output/", 'gemma3_27b-tier_4.json')

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(new_dict_direct, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")

# Emotional Extraction features

In [ ]:
# import split sentences that has been split by LLM
with open('../data/sentences_split_addition_toadd.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

Structure of the data
```python

{
    'file1': ['sentence_1','sentence_2', '....' ,'sentence_n'],
    'file2': ['sentence_1','sentence_2', '....' ,'sentence_n']
}


```

In [ ]:
everything_emotional = {}

error_log = []
for transcript in tqdm(data, desc="Processing entries", unit="entry"):

    raw_output = []
    filename = transcript
    print(filename)

    for i in range(3):
        # Iterate 3 times to ensure consistensy
        mes_1 = "The 8 base emotions in Pultchick's wheel of emotion are Joy, Trust, Fear, Surprise, Sadness, Disgust, Anger and Anticipation. The previous list contains sentences from a transcript from people trying to win 'The most interesting people award', asses the transcript and give a score of between 0 to 100 for the intensity of each of the emotion for each sentence, even if the number is 0; give the answer in the format: 'Sentence 1:..., JOY: XX, TRUST: XX, FEAR:XX, etc.' for each sentence!"
        mes = f"You are a psychiatrist. This is a list of a transcript split into sentences. {data[transcript]}" + mes_1

        message = [
          {"role": "system", "content": "You are a psychiatrist."},
              {
                  "role": "user",
                  "content": mes
              }
        ]
        try:
            result = llm.create_chat_completion(
              message,
                max_tokens = 2000,
                temperature = 0
                
                    )
            # display(Markdown(result["choices"][0]["message"]["content"]))
            raw_output.append(result)
        except Exception as e:
            print("Failed to parse transcript:", e)
            error_log.append(filename)
        
    everything_emotional[filename] = raw_output

# print(everything_emotional)

In [ ]:
with open("../output/gemma3_27b-emotions_valence_island.json", "w") as json_file:
    json.dump(everything_emotional, json_file, indent=4)

# Data preparation

In [ ]:
# This is obtained through Boruta feature selection 
features = ['FS_AVG', 'UFS_AVG', 'FS_MAXHI', 'FS_SVAR', 'UFS_SVAR', 'FS_SPP',
            'FS_APR', 'UFS_SPP', 'UFS_APR', 'trigram_lemma_ttr', 'COCA_spoken_Trigram_Range',
            'COCA_spoken_tri_2_AC','OG_N', 'OG_N_H_CW', 'Freq_N_OG_CW', 'Freq_N_OGH_CW', 'WN_SD', 
            'WN_SD_CW', 'Inflected_Tokens', 'suffix_freq_per_cw', 'affix_freq_per_cw', 
            'mean subset inflectional variety (10)',
            'inflectional TTR (10)', 'ratio_zero_Sadness', 'max_emo_Surprise', 'stdev_Surprise']

In [ ]:
# Inserting definition of each features
with open('../data/new/FEATURES_gemma27b.txt', 'r', encoding='utf-8') as file:
    definition = file.read()


## Importing NLP Feature

In [ ]:
eva_df = pd.read_csv('../data/Feature Set - EVA.csv')
# Step 1: Remove trailing letter (C/S/etc.)
eva_df['id_clean'] = eva_df['PatientID'].str.replace(r'([0-9]{2})[A-Z]$', r'\1', regex=True)

# Step 2: Keep only rows where id ends in '-00'
eva_df = eva_df[eva_df['id_clean'].str.endswith('-00')]

# Step 3: Remove '-00' suffix
eva_df['id_clean'] = eva_df['id_clean'].str.replace(r'-00$', '', regex=True)

# Step 4: Set cleaned ID as index and drop original 'id'
eva_df.set_index('id_clean', inplace=True)
eva_df.drop(columns='PatientID', inplace=True)

eva_df = eva_df[[col for col in features if col in eva_df.columns]]
eva_df

In [ ]:
taales_df = pd.read_csv('../data/Feature Set - TAALES.csv')
# Step 1: Remove trailing letter (C/S/etc.)
taales_df['id_clean'] = taales_df['PatientID'].str.replace(r'([0-9]{2})[A-Z]$', r'\1', regex=True)

# Step 2: Keep only rows where id ends in '-00'
taales_df = taales_df[taales_df['id_clean'].str.endswith('-00')]

# Step 3: Remove '-00' suffix
taales_df['id_clean'] = taales_df['id_clean'].str.replace(r'-00$', '', regex=True)

# Step 4: Set cleaned ID as index and drop original 'id'
taales_df.set_index('id_clean', inplace=True)
taales_df.drop(columns='PatientID', inplace=True)

taales_df = taales_df[[col for col in features if col in taales_df.columns]]
taales_df

In [ ]:
taaco_df = pd.read_csv('../data/Feature Set - TAACO.csv')
# Step 1: Remove trailing letter (C/S/etc.)
taaco_df['id_clean'] = taaco_df['PatientID'].str.replace(r'([0-9]{2})[A-Z]$', r'\1', regex=True)

# Step 2: Keep only rows where id ends in '-00'
taaco_df = taaco_df[taaco_df['id_clean'].str.endswith('-00')]

# Step 3: Remove '-00' suffix
taaco_df['id_clean'] = taaco_df['id_clean'].str.replace(r'-00$', '', regex=True)

# Step 4: Set cleaned ID as index and drop original 'id'
taaco_df.set_index('id_clean', inplace=True)
taaco_df.drop(columns='PatientID', inplace=True)

taaco_df = taaco_df[[col for col in features if col in taaco_df.columns]]
taaco_df

In [ ]:
tammi_df = pd.read_csv('../data/Feature Set - TAMMI.csv')
# Step 1: Remove trailing letter (C/S/etc.)
tammi_df['id_clean'] = tammi_df['PatientID'].str.replace(r'([0-9]{2})[A-Z]$', r'\1', regex=True)

# Step 2: Keep only rows where id ends in '-00'
tammi_df = tammi_df[tammi_df['id_clean'].str.endswith('-00')]

# Step 3: Remove '-00' suffix
tammi_df['id_clean'] = tammi_df['id_clean'].str.replace(r'-00$', '', regex=True)

# Step 4: Set cleaned ID as index and drop original 'id'
tammi_df.set_index('id_clean', inplace=True)
tammi_df.drop(columns='PatientID', inplace=True)

tammi_df = tammi_df[[col for col in features if col in tammi_df.columns]]
tammi_df

## Emotion Feature Engineering 

In [ ]:
with open('../data/emotions_valence_island_new_1.json', 'r', encoding='utf-8') as json_file:
    content = json.load(json_file)

Structure of the data
```python

{
    'file1': [['sentence_1',{'Joy': [20,20,30], 'Trust': [30,40,30], '...'}]['sentence_2',{'Joy': [10,10,10], 'Trust': [50,50,40], '...'}]],
    'file2': [['sentence_1',{'Joy': [45,45,45], 'Trust': [45,45,45], '...'}]['sentence_2',{'Joy': [45,45,45], 'Trust': [45,45,45], '...'}]]
}


```

In [ ]:
# Averaging the emotion scores for each sentence, for each emotion, across the 3 iterations of prompting
averaged_emotion_sentence = {}
for key,value in content.items():
    # print(key, value)
    
    sentence_emotion = {}
    for j in value:
        sentence = j[0]
        emo_list = j[1]
        # print(sentence)
        new_emo_dict = {}
        for emotion in emo_list:
            # print(emotion)
            # print(emo_list[emotion])
            mean = round(statistics.mean(emo_list[emotion]),2)
            if emotion not in sentence_emotion:
                sentence_emotion[emotion] = [mean]
            else:
                sentence_emotion[emotion].append(mean)
    if key not in averaged_emotion_sentence:
        averaged_emotion_sentence[key]= sentence_emotion

averaged_emotion_sentence["L0002"]

In [ ]:
# Averaging the emotion scores for each sentence, for each emotion
averaged_emotion= {}
ka = 0
for key,value in averaged_emotion_sentence.items():
    # print(value)
    whole_emotion = {}
    for key1, value1 in value.items():
        
          
        if key1 not in whole_emotion:
            whole_emotion[key1] = round(statistics.mean(value1),3)
    if key not in averaged_emotion:
        averaged_emotion[key] = whole_emotion
print(averaged_emotion["L0002"])

##### 1. Proportion to 0 of each feat

In [ ]:
proportion_zero = {}
for key,value in averaged_emotion_sentence.items():
    # print(value)
    whole_emotion = {}
    for key1, value1 in value.items():
        name = "ratio_zero_"+key1
        num_0 = 0
        
        for i in reversed(range(len(value1))):
            if value1[i] == 0:
                num_0 += 1
        if key1 not in whole_emotion:
            whole_emotion[name] = round((num_0/len(value1))*100,3)
    if key not in proportion_zero:
        proportion_zero[key] = whole_emotion
print(proportion_zero["L0002"])

##### 2. Lowest non 0 recroded feat (emotion)

In [ ]:
min_emotion_non_zero = {}
for key,value in averaged_emotion_sentence.items():
    # print(value)
    whole_emotion = {}
    for key1, value1 in value.items():
        name = "min_emo_non_0_"+key1
        if averaged_emotion[key][key1] != 0:
            for i in reversed(range(len(value1))):
                if value1[i] == 0:
                    del value1[i]
        if key1 not in whole_emotion:
            whole_emotion[name] = round(min(value1),3)
    if key not in min_emotion_non_zero:
        min_emotion_non_zero[key] = whole_emotion
print(min_emotion_non_zero["L0002"])

##### 3. Maximum recorded for that feat (emotion)

In [ ]:
max_emotion = {}
for key,value in averaged_emotion_sentence.items():
    # print(value)
    whole_emotion = {}
    for key1, value1 in value.items():
        name = "max_emo_"+key1
        if key1 not in whole_emotion:
            whole_emotion[name] = round(max(value1),3)
    if key not in max_emotion:
        max_emotion[key] = whole_emotion
print(max_emotion["L0001"])

##### 4. Average non 0 feature recorded by that feature (emotion)

In [ ]:
averaged_emotion_non_zero = {}
ka = 0
for key,value in averaged_emotion_sentence.items():
    # print(value)
    whole_emotion = {}
    for key1, value1 in value.items():
        name = "avg_non_0_"+key1
        if averaged_emotion[key][key1] != 0:
            for i in reversed(range(len(value1))):
                if value1[i] == 0:
                    del value1[i]
       
        if key1 not in whole_emotion:
            whole_emotion[name] = round(statistics.mean(value1),3)
    if key not in averaged_emotion_non_zero:
        averaged_emotion_non_zero[key] = whole_emotion
print(averaged_emotion_non_zero["L0002"])

##### 5. Std Dev for the emotion for non 0

In [ ]:
stdev_emotion_non_zero = {}
for key,value in averaged_emotion_sentence.items():
    # print(key)
    whole_emotion = {}
    for key1, value1 in value.items():
        name = "stdev_"+key1
        if averaged_emotion[key][key1] != 0:
            for i in reversed(range(len(value1))):
                if value1[i] == 0:
                    del value1[i]
                    
        if key1 not in whole_emotion:
            if len(value1) == 1:
                whole_emotion[name] = 0
                continue
            whole_emotion[name] = round(statistics.stdev(value1),3)
    if key not in stdev_emotion_non_zero:
        stdev_emotion_non_zero[key] = whole_emotion
print(stdev_emotion_non_zero["L0002"])

In [ ]:
df_proportion_zero = pd.DataFrame(proportion_zero).transpose(copy=True)
df_min_emotion_non_zero = pd.DataFrame(min_emotion_non_zero).transpose(copy=True)
df_max_emotion = pd.DataFrame(max_emotion).transpose(copy=True)
df_averaged_emotion_non_zero = pd.DataFrame(averaged_emotion_non_zero).transpose(copy=True)
df_stdev_emotion_non_zero = pd.DataFrame(stdev_emotion_non_zero).transpose(copy=True)
result = df_proportion_zero.join(df_min_emotion_non_zero, how="outer")
result = result.join(df_max_emotion, how="outer")
result = result.join(df_averaged_emotion_non_zero, how="outer")
result = result.join(df_stdev_emotion_non_zero, how="outer")
result


## Adding Ground truth

In [ ]:
# Read the Excel file
uhr_status = pd.read_excel('../data/database.xlsx')

# Display the first few rows of the dataframe

uhr_status['sn'] = uhr_status['sn'].str.replace(r'[A-Za-z]$', '', regex=True)
uhr_status = uhr_status.drop(columns=["caarms_status"])
uhr_status.head()

In [ ]:

result_reset = result.reset_index()
merged = uhr_status.merge(result_reset, left_on='sn', right_on='index')
merged.drop(columns='sn', inplace=True)
merged.set_index('index', inplace=True)
merged

In [ ]:
llm_df = merged[[col for col in features if col in merged.columns]]
llm_df

In [ ]:
llm_eva_df = llm_df.join(eva_df)
llm_eva_tammi_df = llm_eva_df.join(tammi_df)
llm_eva_tammi_taaco_df = llm_eva_tammi_df.join(taaco_df)
llm_eva_tammi_taaco_taales_df = llm_eva_tammi_taaco_df.join(taales_df)
llm_eva_tammi_taaco_taales_df

## Adding ML predictor for LEMon-AID

In [ ]:
ml_prediction = pd.read_csv('../data/XAI Predictions/eva_tax_gemma_27b_best_svm.csv',index_col="sn")
ml_prediction = ml_prediction.replace({1: 'yes', 0: 'no'})
ml_prediction["ML_predicted_label"] = ml_prediction["predicted_label"]
ml_prediction.pop("predicted_label")
ml_prediction

In [ ]:
llm_eva_tammi_taaco_taales_ml_df = llm_eva_tammi_taaco_taales_df.join(ml_prediction["ML_predicted_label"])
llm_eva_tammi_taaco_taales_ml_df

# LEMon

In [ ]:
feature_emo = {}
temperature = 0
for the_id,row in tqdm(llm_eva_tammi_taaco_taales_df.iterrows(), desc="Processing entries", unit="entry", total=len(llm_eva_df)):

    nlp_llm_features = ', '.join(f'{feature} = {row[feature]}' for feature in features)

    mes = "This is definition. "+ definition + "According to the definition, and looking at the next data. Predict whether the person has UHR (Ultra High Risk) of psychosis? Answer in the format of 'ANSWER: *yes or no*' and explain the step by step thought process. " + nlp_llm_features
    message = [
      {"role": "system", "content": "You are psychiatrist."},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 2000,
        temperature = 0.1
            )
    
    feature_emo[the_id] = result["choices"][0]["message"]["content"]
print(feature_emo)

In [ ]:
with open("../output/task3/gemma27b-LEMON-raw.json", "w") as json_file:
    json.dump(feature_emo, json_file, indent=4)

In [ ]:
consolidated_feature_emo_zero_shot = {}
for key,value in feature_emo.items():
    consolidated_feature_emo_zero_shot[key] = extract_first_yes_or_no(value)

In [ ]:
output_file = "../output/llm_eva_prediction/nlp_llm_assisted_prediction-Gemma3-27b.json"

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(consolidated_feature_emo_zero_shot, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")

# LEMon-AID

In [ ]:
feature_emo = {}
temperature = 0
for the_id,row in tqdm(llm_eva_tammi_taaco_taales_df.iterrows(), desc="Processing entries", unit="entry", total=len(llm_eva_df)):
    featureses = features + ["ML_predicted_label"] 

    nlp_llm_features = ', '.join(f'{feature} = {row[feature]}' for feature in featureses)
    
    addition = "A tuned Logistic Regression model predicted the UHR status of the patient and is given in the data as 'ML_predicted_label'. Give a reason why you agree or disagree? And also tell me how much does it influence your decision, in terms of percentage; and in the format of 'ML prediction':xx%."
    mes = "This is definition. "+ definition + "According to the definition, and looking at the next data. Predict whether the person has UHR (Ultra High Risk) of psychosis? Answer in the format of 'ANSWER: *yes or no*' and explain the step by step thought process. " + addition + nlp_llm_features
    
    message = [
      {"role": "system", "content": "You are psychiatrist."},
          {
              "role": "user",
              "content": mes
          }
    ]
    result = llm.create_chat_completion(
      message,
        max_tokens = 2000,
        temperature = 0.1
        
            )
    print(result["choices"][0]["message"]["content"])

    
    feature_emo[the_id] = result["choices"][0]["message"]["content"]
print(feature_emo)

In [ ]:
with open("../output/task3/gemma27b-LEMON_AID-raw.json", "w") as json_file:
    json.dump(feature_emo, json_file, indent=4)

In [ ]:
consolidated_feature_emo_zero_shot = {}
for key,value in feature_emo.items():
    consolidated_feature_emo_zero_shot[key] = extract_first_yes_or_no(value)

In [ ]:
output_file = "../output/task3/gemma27b-LEMON_AID-processed.json"

# Write the combined transcripts to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(consolidated_feature_emo_zero_shot, json_file, ensure_ascii=False, indent=4)

print(f"Transcripts combined and saved to {output_file}")